Nguyễn Đức Minh Mẫn: 2252 0844

Hứa Tấn Sang : 2252 1239

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

#Chuẩn bị dữ liệu mẫu

In [ ]:
# CHọn tâm cụm ban đầu với ma trận hiệp phương sai và các tọa độ như sau
means = [[4, 4], [6, 10], [9, 6]]
cov = [[3, 0], [0, 3]]

# Chọn số mẫu và số vùng
n_samples = 700
n_cluster = 3

# Sinh mẫu
X0 = np.random.multivariate_normal(means[0], cov, n_samples)
X1 = np.random.multivariate_normal(means[1], cov, n_samples)
X2 = np.random.multivariate_normal(means[2], cov, n_samples)

X = np.concatenate((X0, X1, X2), axis = 0) # nối các ma trận mẫu X1, X2, X3 theo hàng

In [ ]:
# Xem phân bố dữ liệu vừa tạo
plt.xlabel('x')
plt.ylabel('y')
plt.plot(X[:, 0], X[:, 1], 'bo', markersize = 5)
plt.plot()
plt.show()

#Tiến hành thuật toán K-means

In [ ]:
# Khởi tạo Kmean cho n_cluster tâm cụm, với tâm cụm được lấy ngẫu nhien từ một điểm trong data đã sinh
def kmeans_init_centers(X, n_cluster):
    return X[np.random.choice(X.shape[0], n_cluster, replace = False)]

In [ ]:
# Hàm này dùng để vẽ dữ liệu lên đồ thị
def kmeans_visualize(X, centers, labels, n_cluster, title):
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(title)
    plt_colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

    for i in range(n_cluster):
        data = X[labels == i]
        plt.plot(data[:, 0], data[:, 1], plt_colors[i] + '^', markersize = 4, label = 'cluster_' + str(i))
        plt.plot(centers[i][0], centers[i][1],  plt_colors[i+4] + 'o', markersize = 10, label = 'center_' + str(i))

    plt.legend()
    plt.show()

In [ ]:
# Phân loại một phần tử mới
'''
Với mỗi điểm trong tập dữ liệu,
cụm của nó là cụm mà khoảng cách từ điểm đó tới tâm là nhỏ nhất
'''
def kmeans_element_label(X, centers):
    D = cdist(X, centers)
    return np.argmin(D, axis = 1)

In [ ]:
# Cập nhật lại tâm của cụm
'''
D(oạn này ta sẽ lấy tất cả các điểm đã được phân vùng,
tính lại giá trị trung bình của phân vùng đó, cho ra center mới
'''
def kmeans_update_centers(X, labels, n_cluster):
    centers = np.zeros((n_cluster, X.shape[1]))
    for k in range(n_cluster):
        # Lấy toàn bộ data của cụm thứ k
        Xk = X[labels == k, :]
        # Lấy trung bình
        centers[k,:] = np.mean(Xk, axis = 0)
    return centers


In [ ]:
'''
Ta kiểm tra liệu rằng tâm cụm của lần thừ i và lần thứ i - 1 có trùng nhau không
nếu có thì ta có thể dừng việc tính toán
'''
def stop_kmeans(centers, new_centers):
  return (set([tuple(a) for a in centers]) == set([tuple(a) for a in new_centers]))

In [ ]:
# Hàm thực hiện K-means
def kmeans(init_centes, init_labels, X, n_cluster):
    centers = init_centes
    labels = init_labels
    times = 0
    while True:
        labels = kmeans_element_label(X, centers)
        kmeans_visualize(X, centers, labels, n_cluster, 'Phân cụm cho dữ liệu sau số lần = ' + str(times + 1))
        new_centers = kmeans_update_centers(X, labels, n_cluster)
        if stop_kmeans(centers, new_centers):
            break
        centers = new_centers
        kmeans_visualize(X, centers, labels, n_cluster, 'Cập nhật tâm cụm sau số lần = ' + str(times + 1))
        times += 1
    return (centers, labels, times)

In [ ]:
init_centers = kmeans_init_centers(X, n_cluster)
print(init_centers)
init_labels = np.zeros(X.shape[0])
kmeans_visualize(X, init_centers, init_labels, n_cluster, 'Khởi tạo K-means với trung tâm là ngẫu nhiên')
centers, labels, times = kmeans(init_centers, init_labels, X, n_cluster)

In [ ]:
# In ra tâm của dữ liệu
print('Center của thuật toán K-means')
print(centers)

print('\nCenter thực sự của bộ dữ liệu')
print(means)

### Đánh giá:
Tâm cụm do K-mean dự đoán khá gần với tâm mà chúng ta đã chọn khi sinh tập dữ liệu.  
=> Thuật toán hoạt động khá chính xác, tuy nhiên cũng một phần là do đã chọn các cụm khá là tách rời nhau ngay từ ban đầu.
